In [1]:
import pandas as pd

Наши новости

In [179]:
pd.set_option('max_rows', 5)
pd.set_option('display.max_colwidth', None)
#pd.set_option('display.float_format', '{:.2f}'.format)

news = pd.read_csv("articles.csv")
print(news.shape)
news.head(10)

(27000, 2)


,doc_id,title
0,6,"Заместитель председателяnправительства РФnСергейnИвановnизбран председателем советаnПБК ЦСКАn. Как сообщаетnпресс-службаnармейцев, такое решение было единогласно принято на первом заседании совета клуба. Основной функцией этого органа будет обсуждение текущего состояния, планирование и утверждение стратегии развития клуба на ближайшую перспективу.nnВ состав совета вошли:nГеоргий Полтавченкоn(полномочный представитель президента в Центральном федеральном округе),nАлексей Седовn(генерал-полковник, начальник службы по защите конституционного строя и борьбе с терроризмомnФСБ РФn),nАлександр Новакn(заместитель министра финансов РФ),nВячеслав Полтавцевn(заместитель генерального директора, руководитель блока по взаимодействию с органами власти, социальной и корпоративной политикиnГМК «Норильский никель»n),nЛеонид Исаковичn(генеральный директор ООО «УК «Спортивные проекты»),nОльга Плешаковаn(генеральный директорn«Трансаэро»n),nАлександр Провоторовn, (генеральный директорnОАО «Ростелеком»n), Андрей Лопатов (экс-игрок ЦСКА, чемпион мира, трехкратный чемпион Европы по баскетболу, председатель правленияnбанка «Клиентский»n),nАндрей Ватутинn(президент ПБК ЦСКА)."
1,4896,"Матч 1/16 финала Кубка России по футболу был приостановлен судьей из-за взрыва пиротехнических снарядов, передает корреспондент «Газеты.Ru». Болельщики выбросили на поле петарды. Судья увел команды с поля в подтрибунное помещение. Динамовцы ушли, а торпедовцы остались у кромки поля. Сообщается, что матч остановлен на пять минут.n«Газета.Ru» ведетnонлайн-трансляциюnэтого матча."
...,...,...
8,4903,"Бывший защитник сборной России Дарюс Каспарайтис поделился ожиданиями от выступления национальной команды на чемпионате мира.n«Все пойдет от лидеров, — заявил Каспарайтис. — В прошлом году, конечно, Малкин много внес в победу сборной. Он играл потрясающе. В этом году Илюха Ковальчук должен взять это на себя, быть лидером. За последнее время Илья очень сильно вырос — и как человек, и как игрок, у него появились очень большие качества лидера».n«Газета.Ru» проведетnонлайн-трансляциюnматча Россия — Латвия.nОзнакомиться с другими материалами, новостями и статистикой можно наnстраницеnсборной России по хоккею."
9,4904,"Полузащитник ЦСКА Зоран Тошич после победы над «Тереком» (n1:0n) в матче 27-го тура чемпионата России заявил, что игроки не делали подарка главному тренеру команды, Леониду Слуцкому, которому 4 мая исполнилось 42 года.n«Мы не делали подарка Слуцкому. Перед игрой наш капитан произнес речь, мы пожелали тренеру победы в чемпионате, — передает слова сербского полузащитника армейцев корреспондент «Газеты.Ru» Данила Жиляев. — После трех не самых удачных игр мы не думали о крупной победе. Просто хотели выиграть. Добились победы, и, можно сказать, сделали еще один шаг к чемпионству. Что касается того, что я хромал, это просто ушиб. Надеюсь, что ничего серьезного. О характере травмы станет известно завтра»."


Загрузим пользователей и списки последних прочитанных новостей

In [3]:
users = pd.read_csv("users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


### 1. Получаем векторные представления новостей

In [5]:
from gensim.corpora.dictionary import Dictionary

In [8]:
#предобработка текстов
import re
import numpy as np
import nltk
from nltk.corpus import stopwords
from razdel import tokenize
import pymorphy2

In [9]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Denic\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
stopword_ru = stopwords.words('russian')
morph = pymorphy2.MorphAnalyzer()
print(stopword_ru[:50])

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть']


In [11]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [165]:
def clean_text(text):
    '''
    очистка текста    
    на выходе очищеный текст    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r"[a-z]", ' ', text)   
    text = re.sub(r"\s{1,}\w{1,3}\s{1,}", ' ', text)     
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [180]:
%%time
#Запускаем очистку текста. Будет долго...
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)

Wall time: 22.6 s


In [181]:
%%time
#Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

Wall time: 2min 15s


In [305]:
news['title'].head(3)

0    [заместитель, председатель, правительство, сергей, иванов, избрать, председатель, совет, цска, сообщать, прессслужба, армеец, решение, единогласно, принять, первый, заседание, совет, клуб, основной, функция, это, орган, обсуждение, текущий, состояние, планирование, утверждение, стратегия, развитие, клуб, близкий, перспектива, состав, совет, войти, георгий, полтавченко, полномочный, представитель, президент, центральный, федеральный, округ, алексей, седов, генералполковник, начальник, служба, защита, конституционный, строй, борьба, терроризм, рф, александр, новак, заместитель, министр, финансы, вячеслав, полтавец, заместитель, генеральный, директор, руководитель, блок, взаимодействие, орган, власть, социальный, корпоративный, политика, норильский, никель, леонид, исакович, генеральный, директор, ук, спортивный, проект, ольга, плешаков, генеральный, директор, трансаэро, александр, провоторов, генеральный, директор, ростелеком, андрей, лопат, эксигрок, цска, чемпион, мир, трёхкратный

А теперь в 3 строчки обучим нашу модель

In [183]:
#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in news['title'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

Запускаем обучение

In [191]:
%%time
from gensim.models import LdaModel
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary)#, passes=10)

Wall time: 25.9 s


In [192]:
from gensim.test.utils import datapath
# Save model to disk.
temp_file = datapath("model.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

Обучили модель

In [193]:
# Create a new corpus, made of previously unseen documents.
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'свой', 'команда', 'матч', 'чемпионат', 'против', 'атланта', 'провести', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'работа', 'сказать', 'заборский', 'получиться', 'забросить', 'быстрый', 'задать', 'хороший', 'темп', 'поединок', 'мочь', 'играть', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'хороший']


[(6, 0.04278572),
 (12, 0.6389042),
 (15, 0.08182067),
 (17, 0.045843787),
 (23, 0.043947395),
 (24, 0.12762272)]

In [223]:
x=lda.show_topics(num_topics=24, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_13: год компания который рост снижение рынок смерть
topic_12: человек мочь который свой жизнь очень это
topic_6: исследование год ракета который пациент это первый
topic_14: экипаж продукция форум цена товар ноябрь годовой
topic_10: женщина тело турция операция мужчина год мозг
topic_7: гражданин год риск блок содержать правительство который
topic_24: таиланд подверженный сдача сбить урожай вырабатывать соль
topic_16: это который страна мочь россия весь говорить
topic_5: статья который университет научный время напомнить год
topic_22: рост это ставка регион год прогноз энергия
topic_11: население год который убийство болезнь космос это
topic_20: год который это россия украина свой писать
topic_0: год место стать самый россиянин весь первый
topic_9: доход школа год фонд сократиться повышение учебный
topic_1: военный который земля американский год человек обнаружить
topic_2: умереть памятник устойчивый мозг скончаться сингапур дворец
topic_4: год который россия также российский реш

Очень неплохо - большинство тем вполне можно описать о чем они

Давайте напишем функцию, которая будет нам возвращать векторное представление новости

In [224]:
#text = news['title'].iloc[0]

def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [225]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.5742,0.0113,...,0.0000,0.0000,0.0000,0.0000,0.0208,0.3678,0.0000,0.0000,0.0000,0.0177
1,4896,0.0000,0.0000,0.0000,0.0483,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0605,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.2787
2,4897,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0430,0.0000,0.0000,...,0.0000,0.0000,0.0539,0.0000,0.0000,0.0000,0.0000,0.0000,0.0636,0.1267
3,4898,0.0909,0.0657,0.0000,0.0000,0.0000,0.0000,0.1572,0.0000,0.0300,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0885,0.0145,0.0000,0.0000,0.0747
4,4899,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.9156,0.0000,0.0000,0.0000,0.0588


Мы получили вектора наших новостей! И даже умеем интерпретировать получившиеся темы.

### Следующий шаг - векторные представления пользователей

In [303]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve
import matplotlib.pyplot as plt

doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))

def get_user_embedding(user_articles_list, stat):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = stat(user_vector, 0)
    return user_vector

def ff(stat = np.mean):
    #Теперь получим эмбединги для всех пользователей и проверим их качество на конкретной downstream-задаче
    user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x,stat), 1)])
    user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
    user_embeddings['uid'] = users['uid'].values
    user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]
    
    #Датасет готов - можно попробовать обучить модель. Загрузим нашу разметку
    target = pd.read_csv("users_churn.csv")
    X = pd.merge(user_embeddings, target, 'left')
    
    #разделим данные на train/test
    X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                    X['churn'], random_state=0)
    
    logreg = LogisticRegression()
    #обучим 
    logreg.fit(X_train, y_train)
    #наши прогнозы для тестовой выборки
    preds = logreg.predict_proba(X_test)[:, 1]
    
    #Рассчитаем Precision, Recall, F_score, roc_auc_score
    precision, recall, thresholds = precision_recall_curve(y_test, preds)
    fscore = (2 * precision * recall) / (precision + recall)
    ix = np.argmax(fscore)
       
    return {"Precision":precision[ix], "Recall":recall[ix], "F-Score":fscore[ix], "ROC-AUC":roc_auc_score(y_test, preds)}

In [327]:
res_mean = ff(np.mean)
res_max_median = ff(np.median)
res_max = ff(np.max)

res_mean["Статистика"] = "maen"
res_max_median["Статистика"] = "median"
res_max["Статистика"] = "max"

q = pd.DataFrame([res_mean,res_max_median,res_max])
q.set_index('Статистика',inplace=True)

q

,Precision,Recall,F-Score,ROC-AUC
Статистика,,,,
maen,0.5462,0.5551,0.5506,0.9006
median,0.6597,0.7755,0.7129,0.9568
max,0.7354,0.8735,0.7985,0.9751
